<a href="https://colab.research.google.com/github/sandeep0412/Data-Imputation-on-Psychology-Data/blob/master/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install PTable

  Stored in directory: /root/.cache/pip/wheels/22/cc/2e/55980bfe86393df3e9896146a01f6802978d09d7ebcba5ea56
Successfully built PTable


In [0]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable

In [0]:
data_num=pd.read_csv("/content/drive/My Drive/data_num.csv",encoding='unicode_escape')
num_samples, column_size = data_num.shape

In [0]:
data_num

In [0]:
x = data_num.isna().sum()
x

gambfalDV                  402
numparticipants_actual    6125
numparticipants           3609
exprace                   3399
age                         16
sunkDV                      14
anchoring1                 982
anchoring2                1060
anchoring3                 717
anchoring4                 735
Ranchori                   982
RAN001                    1060
RAN002                     717
RAN003                     735
Ranch1                     982
Ranch2                    1060
Ranch3                     717
Ranch4                     735
flagdv                       3
Sysjust                     11
Imagineddv                   8
IATexpart                    5
IATexpmath                   3
IATexp.overall               3
anchoring1a               3744
anchoring1b               3582
anchoring2a               3758
anchoring2b               3646
anchoring3a               3481
anchoring3b               3580
anchoring4a               3668
anchoring4b               3411
d_donotu

In [0]:
x = x.sort_values()
x = x.to_frame()
columns_sorted = x.index.values
columns_sorted

array(['previous_session_id', 'omdimc3trt', 'omdimc3rt', 'IATexpmath',
       'flagdv', 'IATexp.overall', 'IATexpart', 'Imagineddv', 'Sysjust',
       'sunkDV', 'age', 'meanerror', 'meanlatency', 'lat12', 'sd2',
       'lat22', 'block2_meanerror', 'sd1', 'block5_meanerror',
       'block3_meanerror', 'block6_meanerror', 'lat11', 'lat21', 'd_art1',
       'd_art2', 'd_art', 'gambfalDV', 'd_donotuse', 'Ranch3', 'RAN002',
       'anchoring3', 'RAN003', 'anchoring4', 'Ranch4', 'Ranch1',
       'anchoring1', 'Ranchori', 'RAN001', 'anchoring2', 'Ranch2',
       'gamblerfallacya', 'gamblerfallacyb', 'exprace', 'anchoring4b',
       'anchoring3a', 'anchoring3b', 'anchoring1b', 'numparticipants',
       'anchoring2b', 'anchoring4a', 'anchoring1a', 'anchoring2a',
       'anchoring3ameter', 'anchoring1akm', 'anchoring1bkm',
       'anchoring3bmeter', 'numparticipants_actual'], dtype=object)

In [0]:
type(list(columns_sorted))

list

In [0]:
data_num_ordered = data_num.filter(columns_sorted, axis=1)
data_num_ordered

In [0]:
#Numerical Values
def standardize_col(col):
    return (col - col.min()) * 1.0 / (col.max() - col.min())

for col in data_num.columns:
    data_num[col] = standardize_col(data_num[col])

In [0]:
data_num['omdimc3trt'][2722]

0.012046206336967917

In [0]:
data_num_ordered.isna().sum()

In [0]:
col_partial = [ 'omdimc3trt', 'omdimc3rt', 'IATexpmath',
       'flagdv', 'IATexp.overall', 'IATexpart', 'Imagineddv', 'Sysjust',
       'sunkDV', 'age', 'meanerror', 'meanlatency', 'lat12', 'sd2',
       'lat22', 'block2_meanerror', 'sd1', 'block5_meanerror',
       'block3_meanerror', 'block6_meanerror', 'lat11', 'lat21', 'd_art1',
       'd_art2', 'd_art', 'gambfalDV', 'd_donotuse', 'Ranch3', 'RAN002',
       'anchoring3', 'RAN003', 'anchoring4', 'Ranch4', 'Ranch1',
       'anchoring1', 'Ranchori', 'RAN001', 'anchoring2', 'Ranch2',
       'gamblerfallacya', 'gamblerfallacyb', 'exprace', 'anchoring4b',
       'anchoring3a', 'anchoring3b', 'anchoring1b', 'numparticipants',
       'anchoring2b', 'anchoring4a', 'anchoring1a', 'anchoring2a',
       'anchoring3ameter', 'anchoring1akm', 'anchoring1bkm',
       'anchoring3bmeter', 'numparticipants_actual']

In [0]:
Z = data_num['previous_session_id'].as_matrix().reshape(-1,1)
Z

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [0]:
Z.shape

(6344, 1)

In [0]:
from sklearn import tree

indexes_dict = {}

for col in col_partial:
#   print(col)
  y  = data_num[col]
#   print(type(y))
  pos = list(y[y.isnull()].index)
#   print(pos)
  indexes_dict[col] = pos
  
  X_train = np.delete(Z, pos, axis=0)
  X_test = Z[pos, :]
  y_train = (y.drop(y.index[pos])).as_matrix().reshape(Z.shape[0]-len(pos),1)

  y_final = y.as_matrix().reshape(Z.shape[0],1)
  
#   print(y.as_matrix().reshape(Z.shape[0],1)[pos[0]])
#   X_test = Z.loc[pos].as_matrix()#.reshape(1,-1)
#   y_train = (y.drop(y.index[pos])).as_matrix(columns = 1)
#   X_train = (Z.drop(Z.index[pos])).as_matrix()
#   X_train = (X_train).as_matrix#.reshape(-1,1)
  
  clf = tree.DecisionTreeRegressor()
  clf = clf.fit(X_train, y_train)
  
  predicted = clf.predict(X_test)
#   print(predicted)
  
  for i in range(len(pos)):
    ind = pos[i]
    y_final[ind] = predicted[i]
  
  Z = np.concatenate((Z,y_final),axis = 1)
#   break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix

In [0]:
Z.shape

(6344, 57)

In [0]:
for l in range(10):
  for col in range(1,Z.shape[1]):
    column_name = col_partial[col-1]
    pos = indexes_dict[column_name]

    X_train = np.delete(Z, col, axis=1)
    X_train = np.delete(X_train, pos, axis=0)
    X_test = np.delete(Z[pos, :], col, axis=1)
    y_train = Z[:, col]
    y_train = np.delete(y_train, pos, axis=0)
    
#     print(X_train.shape)
#     print(X_test.shape)
#     print(y_train.shape)
    
    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(X_train, y_train)

    predicted = clf.predict(X_test)

    for i in range(len(pos)):
      ind = pos[i]
      Z[ind, col] = predicted[i]

    
  

In [0]:
Z.shape

(6344, 57)

In [0]:
data_imputed = pd.DataFrame(Z, columns = columns_sorted)

In [0]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["Column", "N (observed samples)", "Mean (observed samples)", "SD (observed samples)", "Min (observed samples)" , "Max (observed samples)", "N (imputed samples)", "Mean (imputed samples)", "SD (imputed samples)", "Min (imputed samples)" , "Max (imputed samples)"]

In [0]:
data_num1 = pd.read_csv("/content/drive/My Drive/data_num.csv",encoding='unicode_escape')

In [0]:
for col in data_num1.columns:
    data_num1[col] = standardize_col(data_num1[col])

In [0]:
data_imputed_copy = pd.DataFrame(np.nan, index = np.arange(6344), columns = data_num1.columns)
for col in data_num1.columns:

  data_imputed_copy.loc[data_num1[col].isnull(), col] = data_imputed[col]

In [0]:
for col in data_num.columns:
  obs_mean = data_num1[col].mean()
  obs_sd = data_num1[col].std()
  obs_min = data_num1[col].min()
  obs_max = data_num1[col].max()
  
  obs_num = data_num1[col].isna().sum()
  imp_num = num_samples - obs_num
  
  imp_mean = data_imputed_copy[col].mean()
  imp_sd = data_imputed_copy[col].std()
  imp_min = data_imputed_copy[col].min()
  imp_max = data_imputed_copy[col].max()
  
  x.add_row([col, obs_num, obs_mean, obs_sd, obs_min, obs_max, imp_num, imp_mean, imp_sd, imp_min, imp_max])

In [0]:
print(x)

+------------------------+----------------------+-------------------------+-----------------------+------------------------+------------------------+---------------------+------------------------+----------------------+------------------------+-----------------------+
|         Column         | N (observed samples) | Mean (observed samples) | SD (observed samples) | Min (observed samples) | Max (observed samples) | N (imputed samples) | Mean (imputed samples) | SD (imputed samples) | Min (imputed samples)  | Max (imputed samples) |
+------------------------+----------------------+-------------------------+-----------------------+------------------------+------------------------+---------------------+------------------------+----------------------+------------------------+-----------------------+
|       gambfalDV        |         402          |   0.11557210181262266   |   0.1158473994335187  |          0.0           |          1.0           |         5942        |  0.10960397944755204 

In [0]:
data_num1 = pd.read_csv("/content/drive/My Drive/data_num.csv",encoding='unicode_escape')

In [0]:
import random
from math import floor

In [0]:
cnt = 1
for col in data_num_ordered.columns:
  print(col)
#   print(data_num_ordered[col].notnull())
  obs_ind = list(np.where(data_num_ordered[col].notnull())[0])
  print(len(obs_ind))
  
  x_values = []
  
  data_copy = data_num_ordered
  for i in range(floor(0.1*len(obs_ind))):
    leave_ind = (random.choice(obs_ind))
    x_values.append(data_copy[col][leave_ind])
    data_copy[col][leave_ind] = np.nan
  
    
  
  
  
  
  cnt += 1
  if cnt == 2:
    break

previous_session_id
6344


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
type(obs_ind)

list

In [0]:
random.sample([1,2,3,4,5], 3)

[2, 4, 1]

In [0]:
obs_ind

In [0]:
data_num_ordered.isna().sum()

previous_session_id          0
omdimc3trt                   1
omdimc3rt                    1
IATexpmath                   3
flagdv                       3
IATexp.overall               3
IATexpart                    5
Imagineddv                   8
Sysjust                     11
sunkDV                      14
age                         16
meanerror                   44
meanlatency                 44
lat12                       82
sd2                         86
lat22                       87
block2_meanerror            89
sd1                         94
block5_meanerror            96
block3_meanerror            97
block6_meanerror           100
lat11                      103
lat21                      110
d_art1                     124
d_art2                     131
d_art                      159
gambfalDV                  402
d_donotuse                 454
Ranch3                     717
RAN002                     717
anchoring3                 717
RAN003                     735
anchorin

In [0]:
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

def _get_mask(X, value_to_mask):
        if value_to_mask == "NaN":
            # print(np.isnan(X))
            return np.isnan(X)
        else:
            return X == value_to_mask

class MiceImputer(object):

    def __init__(self, missing_values="NaN", strategy="mean", axis=0, verbose=0, copy=True):
        self.missing_values = missing_values
        self.strategy = strategy
        self.axis = axis
        self.verbose = verbose
        self.copy = copy
        self.imp = Imputer(missing_values=self.missing_values, strategy=self.strategy, axis= self.axis, verbose=self.verbose, copy=self.copy)
        self.mask2 = np.zeros([1])

    def _seed_values(self, X):
        self.imp.fit(X)
        return self.imp.transform(X)


    def _process(self, X, column, model_class):
        # Remove values that are in mask
        # mask = np.array(_get_mask(X,self.missing_values)[:, column].T)
#         print()
        mask = _get_mask(X,self.missing_values)
        mask_indices = np.where(mask[:,column]==True)
        
        X_temp = np.array(X)
        index = np.where(mask==True)
        X_temp[index[0],index[1]] = 0
        
#         print("mask")
#         print(mask)
#         print(mask_indices)
        X_data = np.delete(X_temp, mask_indices[0], 0)
#         print("Xdata")
#         print(X_data)

        # Instantiate the model
        model = model_class()

        # Slice out the column to predict and delete the column.
        y_data = X_data[:, column]
        X_data = np.delete(X_data, column, 1)

        # Split training and test data
        X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)

        # Fit the model
        model.fit(X_train, y_train)

        # Score the model
        scores = model.score(X_test, y_test)

        # Predict missing vars
        X_predict = np.delete(X_temp, column, 1)
        y = model.predict(X_predict)

        # Replace values in X with their predictions
        predict_indices = np.array(np.where(self.mask2[:,column]==True))
        X[predict_indices,column] = y[predict_indices]

        return X

    def transform(self, X, model_class=LinearRegression, iterations=10):
        X = np.matrix(X)
        # print('len',len(X.T))
#         print(X)
        self.mask2 = _get_mask(X, self.missing_values)
#         print(self.mask2)
        seeded = self._seed_values(X)
#         print(seeded)

        for i in range(iterations):
            for c in range(len(X.T)):
                X = self._process(seeded, c, model_class)

        # Return X matrix with imputed values
        return X